# Instal·lar GuestAdittions

MUNTAR EL CONTINGUT DE LA ISO, és a dir, el CD Virtual ("/dev/cdrom") al directori "/mnt"

In [ ]:
sudo mount /dev/cdrom /mnt 

Executar l'instal·lador del programa de les GuestAdittions

In [ ]:
cd /mnt
sudo ./VBoxLinuxAdditions.run 

Segurament no us funcionarà la instal·lació, perquè l'instal·lador "VBoxLinuxAdditions" depèn d'altres programes diferents per a que funcioni

\

És a dir, el programa "VBoxLinuxAdditions" té com a dependències altres programes com "bzip2", "tar", "gcc", "make", "perl".

In [ ]:
sudo apt-get update
sudo apt-get install bzip2 tar
sudo apt-get install gcc make perl 

Executar l'instal·lador del programa de les GuestAdittions

In [ ]:
cd /mnt
sudo ./VBoxLinuxAdditions.run 

#### AQUÍ, FER CAPTURA DE PANTALLA DEL RESULTAT DE LA INSTAL·LACIÓ DE LES GUEST ADDITIONS

Si s'han instal·lat les GuestAdittions correctament, reinicieu el sistema operatiu

In [ ]:
sudo reboot

# Creació, configuració i muntatge de la carpeta compartida

Ara anem a crear la carpeta compartida. Per a fer-ho, primer tanqueu la MV

In [ ]:
sudo poweroff
sudo shutdown -h now

Quan estigui la MV tancada, clic dret a la MV i pitja l'opció "paràmetres"


In [ ]:
cp  /etc/samba/smb.conf  /etc/samba/smb.conf.backup
nano /etc/samba/smb.conf

# o per modificar-lo des de zero:

cp  /etc/samba/smb.conf  /etc/samba/smb.conf.backup
rm /etc/samba/smb.conf
nano /etc/samba/smb.conf

- Vés a l'apartat "Carpetes compartides"

- Afegeix una nova carpeta compartida

- En la finestra que apareix escribiu:

-> Selecciona la ruta (del teu ordinador HOST) on vols que es guardi la carpeta virtual. Per exemple, `C:\Users\profe\carpetes-comp-virtualbox`

-> Nom de la carpeta, el que vulgueu, per exemple `ubuntuservercomp`

-> **NO** seleccionar `només lectura` ni `munta automàticament`

-> Punt de muntatge deixeu-lo en blanc


#### AQUÍ, FER CAPTURA DE PANTALLA DE LES CARACTERÍSTIQUES DE LA VOSTRA CARPETA COMPARTIDA

Obre la MV.

Crea una carpeta en el Linux on muntaràs la carpeta compartida. Aquesta carpeta serà el que s'anomena **punt de muntatge**.

In [ ]:
mkdir ~/shared

Munta la carpeta compartida "ubuntuservercomp" al punt de muntatge.

In [ ]:
sudo mount -t vboxsf ubuntuservercomp ~/shared

cd ~/shared

Proveu de crear un fitxer a la carpeta "~/shared" del GUEST i mireu a la carpeta "C:\Users\profe\carpetes-comp-virtualbox" del HOST si hi apareix.

#### AQUÍ, FER CAPTURA DE PANTALLA ON ES VEGI QUE ELS MATEIXOS FITXERS APAREIXEN, EN ELS DIRECTORIS CORRESPONENTS, TANT EN EL HOST COM EN EL GUEST

Si reinicieu la MV la carpeta desapareixerà. Per fer-la permanent, seguiu els passos següents:

## Fer permanent la carpeta compartida (sempre muntada per defecte)


*TODO*